In [81]:
from sklearn.ensemble import VotingRegressor
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor
from sklearn.tree import plot_tree
from sklearn.metrics import roc_auc_score,accuracy_score,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LinearRegression,ElasticNet
from sklearn.neighbors import KNeighborsRegressor

import warnings
warnings.filterwarnings("ignore")



In [2]:
!pip install ISLP

  Using cached ISLP-0.4.0-py3-none-any.whl.metadata (7.0 kB)
  Using cached lifelines-0.30.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached pygam-0.9.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached pytorch_lightning-2.4.0-py3-none-any.whl.metadata (21 kB)
  Using cached torchmetrics-1.5.2-py3-none-any.whl.metadata (20 kB)
  Using cached autograd_gamma-0.5.0-py3-none-any.whl
  Using cached formulaic-1.0.2-py3-none-any.whl.metadata (6.8 kB)
  Using cached progressbar2-4.5.0-py3-none-any.whl.metadata (16 kB)
Using cached ISLP-0.4.0-py3-none-any.whl (3.6 MB)
Using cached lifelines-0.30.0-py3-none-any.whl (349 kB)
Using cached pygam-0.9.1-py3-none-any.whl (522 kB)
Using cached pytorch_lightning-2.4.0-py3-none-any.whl (815 kB)
Using cached torchmetrics-1.5.2-py3-none-any.whl (891 kB)
Using cached formulaic-1.0.2-py3-none-any.whl (94 kB)
Using cached progressbar2-4.5.0-py3-none-any.whl (57 kB)


In [3]:
from ISLP import load_data

In [52]:
Credit = load_data('Credit').set_index('ID')

In [53]:
Credit.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 1 to 400
Data columns (total 11 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   Income     400 non-null    float64 
 1   Limit      400 non-null    int64   
 2   Rating     400 non-null    int64   
 3   Cards      400 non-null    int64   
 4   Age        400 non-null    int64   
 5   Education  400 non-null    int64   
 6   Gender     400 non-null    category
 7   Student    400 non-null    category
 8   Married    400 non-null    category
 9   Ethnicity  400 non-null    category
 10  Balance    400 non-null    int64   
dtypes: category(4), float64(1), int64(6)
memory usage: 27.1 KB


In [54]:
Credit.head(2)

,Income,Limit,Rating,Cards,Age,Education,Gender,Student,Married,Ethnicity,Balance
ID,,,,,,,,,,,
1,14.891,3606,283,2,34,11,Male,No,Yes,Caucasian,333
2,106.025,6645,483,3,82,15,Female,Yes,Yes,Asian,903


In [55]:
X = Credit.drop('Balance',axis=1)
y = Credit['Balance']

In [56]:
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=24, test_size=0.3)

In [57]:
scl = StandardScaler()

In [58]:

ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude='category')  ),
                             (ohe,make_column_selector(dtype_include='category')  ),
                             verbose_feature_names_out=False).set_output(transform='pandas')


In [82]:
el = ElasticNet()
knn = KNeighborsRegressor() 
dt = DecisionTreeRegressor()

In [83]:
pipe_knn = Pipeline([('CT',ct),('KNN',knn)])
pipe_el = Pipeline([('CT',ct),('EL',el)])
pipe_dtr = Pipeline([('CT',ct),('dt',dt)])



In [84]:
pipe_dtr.fit(X_train,y_train)
y_pred = pipe_dtr.predict(X_test)
r2_dtr= r2_score(y_test,y_pred)
print(r2_dtr)

0.8976929234657214


In [85]:
pipe_knn.fit(X_train,y_train)
y_pred = pipe_knn.predict(X_test)
r2_dtr= r2_score(y_test,y_pred)
print(r2_dtr)

0.7899310404076199


In [86]:
pipe_el.fit(X_train,y_train)
y_pred = pipe_el.predict(X_test)
r2_dtr= r2_score(y_test,y_pred)
print(r2_dtr)

0.9057359592636075


In [107]:
voting = VotingRegressor([('DT_P',pipe_dtr),('KNN_P',pipe_knn),('EL_p',pipe_el)],
                        weights=[5,3,5])
voting.fit(X_train,y_train)
y_pred = voting.predict(X_test)
print(r2_score(y_pred,y_test))

0.9092416311967366


### you can also do below method. 

In [108]:

ohe = OneHotEncoder(sparse_output=False,handle_unknown='ignore').set_output(transform='pandas')
ct = make_column_transformer(('passthrough', make_column_selector(dtype_exclude='category')  ),
                             (ohe,make_column_selector(dtype_include='category')  ),
                             verbose_feature_names_out=False).set_output(transform='pandas')

el = ElasticNet()
knn = KNeighborsRegressor() 
dt = DecisionTreeRegressor()

voting = VotingRegressor([('DT',dt),('KNN',knn),('EL',el)], weights=[6,3,6])
pipe_vt = Pipeline([('CT',ct),('VOT',voting)])

pipe_vt.fit(X_train,y_train)
y_pred = pipe_vt.predict(X_test)

print(r2_score(y_test,y_pred))

0.9281970695458998


### GRID SEARCH CV

In [131]:
pipe_vt.get_params()

{'memory': None,
 'steps': [('CT',
   ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002A8D8129460>),
                                   ('onehotencoder',
                                    OneHotEncoder(handle_unknown='ignore',
                                                  sparse_output=False),
                                    <sklearn.compose._column_transformer.make_column_selector object at 0x000002A8D812A660>)],
                     verbose_feature_names_out=False)),
  ('VOT',
   VotingRegressor(estimators=[('DT', DecisionTreeRegressor()),
                               ('KNN', KNeighborsRegressor()),
                               ('EL', ElasticNet())],
                   weights=[6, 3, 6]))],
 'verbose': False,
 'CT': ColumnTransformer(transformers=[('passthrough', 'passthrough',
                                  <sklearn.compose._column_transforme

In [141]:
kfold = StratifiedKFold(n_splits=5,shuffle=True,random_state=24)

params = {  'VOT__DT__min_samples_split':[2,5,10], 'VOT__DT__min_samples_leaf':[1,5,10],'VOT__DT__max_depth':[None,3,5],
         'VOT__EL__alpha': np.linspace(0.001,3,3),
          'VOT__KNN__n_neighbors': [2,5,7],
          'VOT__weights':[[6,3,6],[7,4,7],[9,2,9]]
         }

gcv = GridSearchCV(pipe_vt,param_grid=params,cv=kfold,scoring='r2',verbose=3)

In [142]:
gcv.fit(X,y)

Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV 1/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.946 total time=   0.0s
[CV 2/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.960 total time=   0.0s
[CV 3/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.948 total time=   0.0s
[CV 4/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neighbors=2, VOT__weights=[6, 3, 6];, score=0.953 total time=   0.0s
[CV 5/5] END VOT__DT__max_depth=None, VOT__DT__min_samples_leaf=1, VOT__DT__min_samples_split=2, VOT__EL__alpha=0.001, VOT__KNN__n_neigh

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=24, shuffle=True),
             estimator=Pipeline(steps=[('CT',
                                        ColumnTransformer(transformers=[('passthrough',
                                                                         'passthrough',
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000002A8D8129460>),
                                                                        ('onehotencoder',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse_output=False),
                                                                         <sklearn.compose._colu...
                                                                     DecisionTreeRegressor()),
                                                                    ('KNN',
                                                                     KNeighborsRegressor()),
                                                                    ('EL',
                                                                     ElasticNet())],
                                                        weights=[6, 3, 6]))]),
             param_grid={'VOT__DT__max_depth': [None, 3, 5],
                         'VOT__DT__min_samples_leaf': [1, 5, 10],
                         'VOT__DT__min_samples_split': [2, 5, 10],
                         'VOT__EL__alpha': array([1.0000e-03, 1.5005e+00, 3.0000e+00]),
                         'VOT__KNN__n_neighbors': [2, 5, 7],
                         'VOT__weights': [[6, 3, 6], [7, 4, 7], [9, 2, 9]]},
             scoring='r2', verbose=3)

In [143]:
print(gcv.best_params_)


{'VOT__DT__max_depth': None, 'VOT__DT__min_samples_leaf': 1, 'VOT__DT__min_samples_split': 5, 'VOT__EL__alpha': 0.001, 'VOT__KNN__n_neighbors': 5, 'VOT__weights': [9, 2, 9]}


In [144]:
print(gcv.best_score_)

0.956102351968882


In [146]:
pd_cv = pd.DataFrame(gcv.cv_results_)

In [148]:
pd_cv.to_csv('Credit_ISLP_Voting_regressor.csv')

In [150]:
pd_cv.shape

(729, 19)